<a href="https://colab.research.google.com/github/Madhanmetha/ML/blob/main/TeachAgent_LangChain_Exercise_No_7_Working.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To use the Google Generative AI API, you'll need an API key. If you don't already have one, create a key in Google AI Studio.

In Colab, add the key to the secrets manager under the "🔑" in the left panel. Give it the name `GOOGLE_API_KEY`. Then pass the key to the SDK:

In [16]:
!pip install -q langchain langchain-google-genai google-generativeai

import google.generativeai as genai
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType, Tool

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

Now we will define the tools and initialize the agent with a Generative AI model.

In [17]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_google_genai import ChatGoogleGenerativeAI

# Calculator tool
def calculator_tool(query):
    try:
        return str(eval(query))
    except:
        return "Invalid calculation."

# Lookup tool
knowledge_db = {
    "largest planet": "Jupiter is the largest planet in our Solar System.",
    "capital of japan": "Tokyo is the capital of Japan.",
    "einstein born": "Albert Einstein was born in 1879."
}

def lookup_tool(query):
    query_lower = query.lower()
    for key, info in knowledge_db.items():
        if key in query_lower:
            return info
    return "I don't know."

tools = [
    Tool(name="Calculator", func=calculator_tool, description="Perform arithmetic calculations."),
    Tool(name="Lookup", func=lookup_tool, description="Find factual information from a small knowledge base.")
]

# Initialize the Generative AI model
# The model 'gemini-pro' was not found, changing to 'gemini-pro-vision'.
# You might need to change this model name if 'gemini-pro-vision' is also not available in your environment.
# Updating to gemini-1.5-flash as gemini-pro-vision is deprecated.
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0, google_api_key=GOOGLE_API_KEY)


# Initialize and run a LangChain agent using custom tools and the Generative AI LLM
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

question = "What is the capital of Japan and what is 15 + 227?"
response = agent.invoke({
    "input": question,
    "chat_history": []
})
print("Final Answer:", response)



> Entering new AgentExecutor chain...
Thought: I need to find the capital of Japan and calculate the sum of 15 and 227.  I'll use Lookup for the capital and Calculator for the sum.

Action: Lookup
Action Input: What is the capital of Japan?
Observation: Tokyo is the capital of Japan.
Thought:Thought: Now I need to calculate 15 + 227.
Action: Calculator
Action Input: 15 + 227
Observation: 242
Thought:I now know the final answer
Final Answer: The capital of Japan is Tokyo and 15 + 227 = 242.

> Finished chain.
Final Answer: {'input': 'What is the capital of Japan and what is 15 + 227?', 'chat_history': [], 'output': 'The capital of Japan is Tokyo and 15 + 227 = 242.'}


In [19]:
# Directly use the defined tools to answer parts of the question
capital_of_japan = lookup_tool("capital of Japan")
calculation_result = calculator_tool("15 + 227")

print("Using the Lookup tool:")
print(f"What is the capital of Japan? {capital_of_japan}")
print("\nUsing the Calculator tool:")
print(f"What is 15 + 227? {calculation_result}")

Using the Lookup tool:
What is the capital of Japan? Tokyo is the capital of Japan.

Using the Calculator tool:
What is 15 + 227? 242
